In [18]:
from copy import deepcopy

import numpy as np
from numpy.random import randint

from utilities import create_laplace_profile, profile_most_probable_kmer, score_motifs

In [13]:
t=10
k=4
randint(0, size=10, high=t-k+1)

array([3, 5, 4, 3, 3, 0, 2, 0, 4, 4])

In [14]:
def select_random_motifs(dna, k):
    t = len(dna)
    motif_start_indices = randint(0, size=t, high=len(dna[0])-k+1)
    motifs = []
    for i, start in enumerate(motif_start_indices):
        motifs.append(dna[i][start:start+k])
    return motifs
 

In [23]:
def randomized_motif_search_inner(dna, k):
    motifs = select_random_motifs(dna, k)
    best_motifs = deepcopy(motifs)
    best_score = score_motifs(best_motifs)
    while True:
        profile = create_laplace_profile(motifs)
        motifs = []
        for dna_string in dna:
            motif = profile_most_probable_kmer(dna_string, k, profile)
            motifs.append(motif)
        score = score_motifs(motifs)
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
        else:
            return best_motifs
        
def randomized_motif_search(dna, k, n_iter=1000):
    best_motifs = randomized_motif_search_inner(dna, k)
    best_score = score_motifs(best_motifs)
    for i in range(n_iter-1):
        motifs = randomized_motif_search_inner(dna, k)
        score = score_motifs(motifs)        
        if score < best_score:
            best_motifs = deepcopy(motifs)
            best_score = score
    return best_motifs

In [24]:
k = 8 
dna = [
    "CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
    "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
    "TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
    "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
    "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"
]

In [26]:
expected = [
    "TCTCGGGG",
    "CCAAGGTG",
    "TACAGGCG",
    "TTCAGGTG",
    "TCCACGTG"
]

assert randomized_motif_search(dna, k) == expected